<div style="text-align: center;">

# SCRIPT POUR LE SCRAPPING DE VIDEO YOUTUBE (FILMS BURKINBE ) 

## SOLLYWOOD - Partie 1 (SCRAPPING)
    
<div style="display: inline-block; text-align: center; margin-top: 20px;">
    <div style="display: inline-block; margin-right: 20px;">
        <img src="https://www.guru.com/blog/wp-content/uploads/2022/06/legal-to-scrape-data-from-websites.jpg" alt="Image 1" style="width: 250px; height: auto;">
    </div>
    <div style="display: inline-block;">
        <img src="https://scontent-cdg4-2.xx.fbcdn.net/v/t39.30808-6/368609641_620973146817213_2768711167502916787_n.jpg?_nc_cat=107&ccb=1-7&_nc_sid=833d8c&_nc_ohc=jFnyq5UjCCQQ7kNvgE0pDOt&_nc_ht=scontent-cdg4-2.xx&oh=00_AYDkfpr-909u0Aqf9F8j8qK__VwG770C5fidpqo5CTE-xQ&oe=6696527F" alt="Image 2" style="width: 250px; height: auto;">
    </div>
</div>

</div>

## Contexte
Ayant pour projet de lancer une plateforme de streaming de films burkinabé, il me fallait avoir une vision globale sur l'état du cinéma burkinabé, son audience, etc. Pour cela, je devais obtenir des données à ce sujet. Ne sachant pas qui contacter pour obtenir ces données et étant Data Engineer de formation, j'ai décidé de créer cette base de données moi-même afin de l'étudier et de la partager avec tous ceux qui en ont besoin.

## Collecte de Données
Pour obtenir des données, quoi de mieux que YouTube, où de nombreux films burkinabé sont disponibles (illégalement ou légalement). C'était pour moi l'occasion de me faire une idée de l'audience.

### Étape 1 : Scrapping des Vidéos
Pour étudier ces films, j'ai pris en compte les informations suivantes :
* **Le Titre**
* **La Durée**
* **Le Nombre de Vues**
* **Le Nombre de Likes**
* **La Description**
* **Les Commentaires**

### Pourquoi les Commentaires ?
L'étude des commentaires me permettra d'avoir le ressenti des internautes sur le film.


In [1]:
#INITIALISATION DES PACKAGES

from concurrent.futures import ThreadPoolExecutor, as_completed
from selenium import webdriver
from selenium.webdriver.firefox.service import Service as FirefoxService
from webdriver_manager.firefox import GeckoDriverManager
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import re


# Chemin vers votre profil Firefox avec uBlock Origin installé
firefox_profile_path = 'Entrer le chemin vers votre profil firefox avec un boquer de pub actif'

In [2]:
#Fonction pour formater le nombre de vue
def format_view_count(view_count_str):
    # Utiliser une expression régulière pour extraire les chiffres et les regrouper
    view_count_str = re.sub(r'\D', '', view_count_str)
    return f'{int(view_count_str):,}'.replace(',', ' ')


In [3]:
#Fonction pour initialiser le driver
def init_driver():
    firefox_options = webdriver.FirefoxOptions()
    #Utilisation du profil car j'avais besoin d'un bloqueur de pub qui etait deja installé sur mon profil firefox
    firefox_options.profile = firefox_profile_path
    #firefox_options.add_argument("--headless")  # Activer le mode headless pour améliorer les performances
    driver = webdriver.Firefox(
        service=FirefoxService(GeckoDriverManager().install()), 
        options=firefox_options
    )
    return driver



In [4]:
#Fonction pour recuperer les urls des videos
def get_video_urls(driver, scroll_pause_time=2, max_scrolls=5000, max_urls=5000):
    video_urls =[]
    # Attendre que la page se charge
    time.sleep(5)
    # Maximiser la fenêtre du navigateur
    driver.maximize_window()
    time.sleep(2)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    scrolls = 0
    urls_collected = 0
    
    while scrolls < max_scrolls and urls_collected < max_urls:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(scroll_pause_time)
        
        video_links = driver.find_elements(By.XPATH, '//*[@id="video-title"]')
        new_urls = [link.get_attribute('href') for link in video_links]
        new_urls = list(filter(None, new_urls))  # Remove any None values
        
        for url in new_urls:
            if urls_collected < max_urls:
                video_urls.append(url)
                urls_collected += 1
            else:
                break
        
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        scrolls += 1
        
    return video_urls
     

In [5]:
# Fonction pour scraper les informations d'une vidéo
def scrape_video_info(url):
    driver = init_driver()
    video_info = {}
   
    
    try:
        driver.get(url)
        
        time.sleep(10)

        WebDriverWait(driver, 15).until(
            EC.visibility_of_element_located((By.CSS_SELECTOR, '#title h1 yt-formatted-string'))
        )
        
        driver.find_element(By.ID, 'description-inline-expander').click()

        # Récupérer le titre de la vidéo
        video_info["url"] = url
        video_info["title"] = driver.find_element(By.CSS_SELECTOR, '#title h1 yt-formatted-string').text.strip()

        # Récupérer la durée de la vidéo
        video_info["duration"] = driver.find_element(By.CSS_SELECTOR, 'span.ytp-time-duration').text.strip()

        # Récupérer le nombre de vues de la vidéo
        view_count = driver.find_element(By.XPATH, '//*[@id="info"]/span[1]').text.strip()
        video_info["views"] = format_view_count(view_count)
        #video_info["views"] = driver.find_element(By.XPATH, '//*[@id="info"]/span[1]').text.strip()
        
        # Récupérer la date de publication de la vidéo
        video_info["date"] = driver.find_element(By.XPATH, '//*[@id="info"]/span[3]').text.strip()

        # Récupérer le nom de la chaîne
        video_info["channel"] = driver.find_element(By.XPATH, '//*[@id="text"]/a').text.strip()

        # Récupérer les likes de la vidéo
        video_info["like"] = driver.find_element(By.XPATH, '//*[@id="top-level-buttons-computed"]/segmented-like-dislike-button-view-model/yt-smartimation/div/div/like-button-view-model/toggle-button-view-model/button-view-model/button/div[2]').text.strip()

        # Récupérer la description de la vidéo
        driver.find_element(By.ID, 'description-inline-expander').click()
        video_info["description"] = driver.find_element(By.XPATH, '//*[@id="description-inline-expander"]/yt-attributed-string').text.strip()

        #défiler pour charger les commentaires
        driver.execute_script("window.scrollTo(0, 1000);")
        for i in range(8):
            time.sleep(2)
            driver.execute_script("window.scrollTo(0, 10000);")

        #récupéreration des commentaires
        comment_elements = driver.find_elements(By.CSS_SELECTOR, '#content #content-text')
        video_info["comments"] = [comment.text for comment in comment_elements]

    except Exception as e:
        print(f'Erreur lors de l\'extraction des informations de la vidéo {url}: {e}')
    
    finally:
        if driver:
            driver.quit()
    
    return video_info


* J'ai fait ici le choix de ne scrapper les urls des videos a scrapper a partir de 3 recherches. 
* LSi vous voulez l'adapter a votre situation sur youtube entre votre recherche par exemple "Film Français" valider et ensuite recuperer le liens de la recherche et entre le dans playlist_urls

In [6]:
playlist_urls = [
    'https://www.youtube.com/results?search_query=film+burkinab%C3%A9',
    'https://www.youtube.com/results?search_query=film+complet+burkinab%C3%A9',
    'https://www.youtube.com/results?search_query=film+burkina'
    
]

# liste pour stocker toutes les URLs
all_video_urls = []


In [7]:
# Initialisation du le driver
driver = init_driver()

# Boucle pour parcourir chaque URL de la playlist
for url in playlist_urls:
    driver.get(url)
    video_urls = get_video_urls(driver)
    all_video_urls.extend(video_urls)

driver.quit()


print("video url bon")

video url bon


In [8]:
len(all_video_urls)

15000

* Il arrive qu'il y ait des doublons donc pour se rassurer on supprime

In [9]:
#Suppression doublon
all_video_urls_unique = list(set(all_video_urls))
len(all_video_urls_unique)

1239

### Pour faciliter le traitement separe en paquet de 100

In [10]:
def split_list(input_list, chunk_size):
    #création des sous-listes de taille chunk_size
    return [input_list[i:i + chunk_size] for i in range(0, len(input_list), chunk_size)]

# Diviser la liste en sous-listes de 100 éléments
chunked_lists = split_list(all_video_urls_unique, 100)

In [11]:
chunked_lists[0][99]

'https://www.youtube.com/watch?v=6sBwJVaCvQE&pp=ygUXZmlsbSBjb21wbGV0IGJ1cmtpbmFiw6k%3D'

In [12]:
chunked_lists[1][99]

'https://www.youtube.com/watch?v=551diBfP63Q&pp=ygUMZmlsbSBidXJraW5h'

In [13]:
videos_data = []

### J'ai decidé plus haut d'évoluer par paquet de 100 donc pour traiter ces paquet j'ai mis en place un système en parelelisme de 10 instances pour arriver a les traiter plus rapidement



In [33]:
for i in range(0,len(chunked_lists)) :
    with ThreadPoolExecutor(max_workers=10) as executor:
        # Démarrer les tâches et récupérer les futures
        futures = [executor.submit(scrape_video_info, url) for url in chunked_lists[i]]

        # Attendre que toutes les tâches soient terminées
        for future in as_completed(futures):
            try:
                video_info = future.result()
                # Traiter les informations récupérées (par exemple, stocker dans une base de données)
                #print("toto")
                #print(video_info)
                videos_data.append(video_info)
            except Exception as exc:
                print(f'Une erreur s\'est produite : {exc}')
    print(i)           
    time.sleep(10)

Erreur lors de l'extraction des informations de la vidéo https://www.youtube.com/shorts/tqykm81mrfk: Message: 

10
Erreur lors de l'extraction des informations de la vidéo https://www.youtube.com/shorts/zRgjj9k7w3c: Message: 

Erreur lors de l'extraction des informations de la vidéo https://www.youtube.com/shorts/nzCUnWImZxI: Message: 

11
12


In [34]:
len(videos_data)

1239

In [35]:
videos_data[199]

{'url': 'https://www.youtube.com/watch?v=551diBfP63Q&pp=ygUMZmlsbSBidXJraW5h',
 'title': "L'As du lycée - S01 - Episode 40",
 'duration': '12:09',
 'views': '3 364 592',
 'date': 'Sortie le 23 oct. 2020',
 'channel': 'FasoFilms TV',
 'like': '20 k',
 'description': 'L’As du lycée \nSaison 1 - Episode 40 : La révolte de Kouka\nRéalisateur : Missa Hébié\n\nHumour # Argent # Pauvreté...',
 'comments': []}

In [36]:
#Creation de dataFrame
import pandas as pd 
    
df = pd.DataFrame(videos_data,columns=['url','title','duration','views','date','channel','like','description','comments'])
df

,url,title,duration,views,date,channel,like,description,comments
0,https://www.youtube.com/watch?v=7egUxmcZJ2M&pp...,La Vengeance de Moussa Koffe - Film Complet,1:15:46,999 349,10 sept. 2012,AfricaShows: 1ère chaîne de divertissement en ...,"3,6 k",Rejoignez la communauté : http://bit.ly/PFvmXV...,[Qui est là après le décès de Koutoko \nQu'All...
1,https://www.youtube.com/watch?v=Mu9YXOgF5zo&pp...,Sondo 2 (film burkinabè),1:44:33,32 018,Sortie le 11 avr. 2022,KAFMAN GROUP,458,un film réalise par Christian M. K. KAFO et pr...,[]
2,https://www.youtube.com/watch?v=0-mPPirjHok&pp...,Film Burkinabe: une petite fille intelligente ...,0:34,1 540,25 avr. 2020,daksoum2007,5,,[]
3,https://www.youtube.com/watch?v=Qe9ugTqVBqw&pp...,Kato Kato,1:12:42,177 536,8 mai 2015,AfricaShows: 1ère chaîne de divertissement en ...,1 k,Rejoignez la communauté : http://bit.ly/PFvmXV...,[Ohh l'époque m mank la première fois q j'ai r...
4,https://www.youtube.com/watch?v=A8SdY35euXU&pp...,capitaine Ibrahim Traoré sa dresse aux hommes ...,12:15,51 711,10 juil. 2024,ismael info,1\n2\n k,,"[J'ai n'ai plus de mots pour vous, le présiden..."
...,...,...,...,...,...,...,...,...,...
1234,https://www.youtube.com/watch?v=BIweQKIcOzw&pp...,Commissariat de Tampy - S01- E01,20:00,1 223 276,Sortie le 29 mai 2020,FasoFilms TV,8 k,Commissariat de Tampy \nSaison 1 - Episode 01 ...,"[2024 qui est là 🇹🇬, On remerciera jamais asse..."
1235,https://www.youtube.com/watch?v=mwWTdUuxsLA&pp...,CRIME DU COEUR ! un film de Hounsou Euloge - F...,1:31:30,36 317,26 mai 2024,AFRICAKULTURE+ TV,275,► Abonne-toi à la chaine MAINTENANT c'est grat...,[Faudrait revoir la mise en scène de fiction ...
1236,https://www.youtube.com/watch?v=l3NzwFftTO4&pp...,Étalons : L'appel à la cohésion du Capitaine I...,9:09,152 491,7 juil. 2024,Faso7 TV,"5,5 k","Un match de gala a opposé, le 29 juin 2024, l’...",[Je vous jalouses hein les burkinabè vous avez...
1237,https://www.youtube.com/watch?v=g4QTBcnL-zE&pp...,POLYGAMIE D'UN HOMME PAUVRE - FILM COMPLET VER...,1:16:56,606 515,11 mars 2019,AfricaShows: 1ère chaîne de divertissement en ...,"2,6 k",rejoignez la communauté : http://bit.ly/PFvmXV...,[Ses deux complets là ... ses deux complets ja...


In [37]:
# Exportation du DataFrame en fichier CSV
csv_file_path = 'film_burkinabe_yt_.csv'
df.to_csv(csv_file_path, index=False, encoding='utf-8')

print(f'Les données des vidéos ont été exportées vers {csv_file_path}')

Les données des vidéos ont été exportées vers film_burkinabe_yt_.csv
